In [4]:
import os
import numpy as np
from PIL import Image
import sqlite3
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

INCLUDED_EXTENTION = [".png", ".jpg"]

dbname = 'images.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('DROP TABLE image_info')
cur.execute('CREATE TABLE image_info(id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
conn.commit()
conn.close()

conn = sqlite3.connect(dbname)
cur = conn.cursor()
filenames = sorted(os.listdir('handwriting_pics'))
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
conn.commit()
cur.close()
conn.close()

conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('SELECT * FROM image_info')
pics_info = cur.fetchall()
cur.close()
conn.close()

img_test = np.empty((0, 64))
for pic_info in pics_info:
    filename = pic_info[1]
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    img = Image.open(f'handwriting_pics/{filename}').convert('L')
    img_data256 = 255 - np.array(img.resize((8, 8)))

    min_bright = img_data256.min()
    max_bright = img_data256.max()
    img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
    img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]

digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
logreg = LogisticRegression(max_iter=2000)
logreg_model = logreg.fit(X_train, y_train)

X_true = []
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    X_true = X_true + [int(filename[:1])]
X_true = np.array(X_true)
pred_logreg = logreg_model.predict(img_test)

print('손글씨 문자의 판별 결과')
print('관측 결과:', X_true)
print('예측 결과:', pred_logreg)
print('정답률:', logreg_model.score(img_test, X_true))

OperationalError: no such table: image_info

In [ ]:
INCLUDED_EXTENTION = [".png", ".jpg"]

# 이미지가 들어있는 폴더를 지정하고, 내용의 파일명을 취득
# images.db를 신규 작성. images.db가 이미 존재하고 있으면 접속.
dbname = 'images.db'
# 데이터베이스로의 커넥션 오브젝트 작성
conn = sqlite3.connect(dbname)
# sqlite를 조작하는 커서 오브젝트를 작성
cur = conn.cursor()
# 데이터베이스의 초기화
cur.execute('DROP TABLE image_info')
# image_info라는 table을 작성
cur.execute('CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
# 데이터베이스에 커밋하고, 변경을 보존
conn.commit()
conn.close()

# 데이터베이스에 이미지의 파일명을 삽입
conn = sqlite3.connect(dbname)
cur = conn.cursor()
filenames = sorted(os.listdir('handwriting_pics'))
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
conn.commit()
cur.close()
conn.close()

# table의 내용을 취득
conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('SELECT * FROM image_info')
# fetchall()을 사용해서 내용을 전부 취득
pics_info = cur.fetchall()
cur.close()
conn.close()

In [ ]:
img_test = np.empty((0, 64))
#　폴더 내의 전 이미지를 데이터화
for pic_info in pics_info:
    filename = pic_info[1]
    #　이미지 파일을 취득, 그레이스케일로 하여 크기 변경
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    img = Image.open(f'handwriting_pics/{filename}').convert('L')
    img_data256 = 255 - np.array(img.resize((8, 8)))

    # 이미지 데이터 내의 최솟값이 0, 최댓값이 16이 되도록 계산
    min_bright = img_data256.min()
    max_bright = img_data256.max()
    img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
    # 가공한 이미지 데이터의 배열을 합친다
    img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]

In [ ]:
# 지도 데이터로부터의 학습
# sklearn의 데이터셋으로부터 취득, 목적 변수 X와 설명 변수 y로 나눈다
digits = load_digits()
X = digits.data
y = digits.target
# 지도 데이터와 테스트 데이터로 나눈다
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
# 로지스틱 회귀의 모델을 작성하고, 지도 데이터를 사용해서 학습시킨다
logreg = LogisticRegression(max_iter=2000)
logreg_model = logreg.fit(X_train, y_train)

# 이미지 데이터의 판별
# 이미지 데이터의 정답을 배열로 한다
X_true = []
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    X_true = X_true + [int(filename[:1])]
X_true = np.array(X_true)

# 로지스틱 회귀의 학습 완료 모델에 이미지 데이터를 넣고, 판별한다
pred_logreg = logreg_model.predict(img_test)

print('손글씨 문자의 판별 결과')
print('관측 결과:', X_true)
print('예측 결과:', pred_logreg)
print('정답률:', logreg_model.score(img_test, X_true))

In [ ]:
import os
import numpy as np
from PIL import Image
import sqlite3
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
INCLUDED_EXTENTION = [".png", ".jpg"]
dbname = 'images.db'
dir_name = 'handwriting_pics'

def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """손글씨 문자 이미지가 놓여 있는 패스로부터 파일명을 취득하고, 리스트를 작성"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def create_table(dbname):
    """테이블을 작성하는 함수"""
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    cur.execute('DROP TABLE image_info')
    cur.execute( 'CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
    conn.commit()
    conn.close()
    print("table is successully created")

def insert_filenames(dbname, dir_name):
    """손글씨 문자 이미지의 파일명을 데이터베이스에 보존"""
    filenames = load_filenames(dir_name)
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    for filename in filenames:
        cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
    conn.commit()
    cur.close()
    conn.close()
    print("image file names are successully inserted")

def extract_filenames(dbname):
    """손글씨 문자 이미지의 파일명을 데이터베이스로부터 취득"""
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    cur.execute( 'SELECT * FROM image_info')
    filenames = cur.fetchall()
    cur.close()
    conn.close()
    return filenames

create_table(dbname)
insert_filenames(dbname, dir_name)
extract_filenames(dbname)

In [ ]:
# p425, 아웃풋을 확인 필요
def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """손글씨 문자 이미지가 놓여 있는 패스로부터 파일명을 취득하고, 리스트를 작성하는 함수"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def get_grayscale(dir_name):
    """읽어 들인 손글씨 문자 이미지의 색을 그레이스케일로 변환하는 함수(그레이스케일은 색의 농담의 명함을 나누는 방법)"""
    filenames = load_filenames(dir_name)
    for filename in filenames:
        img = Image.open(f'{dir_name}/{filename}').convert('L')
        yield img

def get_shrinked_img(dir_name):
    """이미지 크기를 8×8 픽셀의 크기로 통일하고, 밝기도 16계조의 그레이스케일로 흑백으로 변환하는 함수"""
    img_test = np.empty((0, 64))
    crop_size = 8
    for img in get_grayscale(dir_name):
        img_data256 = 255 - np.array(img.resize((crop_size, crop_size)))
        min_bright, max_bright = img_data256.min(),  img_data256.max()
        img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
        img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]
    return img_test

img_test = get_shrinked_img(dir_name)
get_shrinked_img(dir_name)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """손글씨 문자 이미지가 놓여 있는 파일명을 취득하고, 리스트를 작성"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def create_logreg_model():
    """로지스틱 회귀의 학습 완료 모델을 생성"""
    digits = load_digits()
    X = digits.data
    y = digits.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    logreg = LogisticRegression(max_iter=2000)
    logreg_model = logreg.fit(X_train, y_train)
    return logreg_model

def evaluate_probs(dir_name, img_test, logreg_model):
    """테스트 데이터를 이용하여 로지스틱 회귀의 학습 완료 모델의 아웃풋을 평가"""
    filenames = load_filenames(dir_name)
    X_true = [int(filename[:1]) for filename in filenames]  
    X_true = np.array(X_true)
    pred_logreg = logreg_model.predict(img_test)
    
    print('손글씨 문자의 판별 결과')
    print('관측 결과:', X_true)
    print('예측 결과:', pred_logreg)
    print('정답률:', logreg_model.score(img_test, X_true))
    return "Propability calculation is successfully finished"

logreg_model = create_logreg_model()
evaluate_probs(dir_name, img_test, logreg_model)

In [ ]:
import pickle
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

logreg = LogisticRegression(max_iter=2000)
model = logreg.fit(X_train, y_train)
with open('model.pickle', mode='wb') as fp:
    pickle.dump(model, fp)